#Load

In [10]:
from pathlib import Path
import pandas as pd
import urllib.request


def load_housing_data():
  sold_file=Path("datasets/sold_melbourne.csv")
  if not sold_file.is_file():
    url="https://raw.githubusercontent.com/mmercan/handson-ml3/refs/heads/main/data/housing/sold_melbourne.csv"
    urllib.request.urlretrieve(url,sold_file)

    house=pd.read_csv(sold_file)
    house['price']=housing['price'].str.replace('$', '', regex=False)
  return housing


housing = load_housing_data()
housing.info()
housing.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105120 entries, 0 to 105119
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   latitude       105119 non-null  float64
 1   longitude      105119 non-null  float64
 2   streetAddress  105120 non-null  object 
 3   suburb         105120 non-null  object 
 4   postcode       105120 non-null  int64  
 5   region         99878 non-null   object 
 6   bedrooms       105120 non-null  int64  
 7   bathrooms      105120 non-null  int64  
 8   parkingSpaces  105120 non-null  int64  
 9   propertyType   105120 non-null  object 
 10  price          105120 non-null  object 
 11  listingId      105120 non-null  int64  
 12  title          105120 non-null  object 
 13  dateSold       105120 non-null  object 
 14  modifiedDate   105120 non-null  object 
dtypes: float64(2), int64(5), object(8)
memory usage: 12.0+ MB


,latitude,longitude,streetAddress,suburb,postcode,region,bedrooms,bathrooms,parkingSpaces,propertyType,price,listingId,title,dateSold,modifiedDate
0,-38.276067,144.485488,6 Corymbia Circuit,Barwon Heads,3227,bellarine_peninsula,3,2,2,house,"1,255,000",129714950,Luxurious Coastal Lifestyle Awaits,2018-11-06,2018-11-06T02:00:19Z
1,-38.238159,144.545576,9 La Bella Street,Ocean Grove,3226,bellarine_peninsula,4,2,2,house,"725,000",129219294,Laid Back Family Living In An Idyllic Location,2018-11-06,2018-11-06T04:01:41Z
2,-37.638387,144.922701,26 Greensted Grove,Roxburgh Park,3064,melbourne___northern_region,4,2,2,house,"670,000",129517974,Oversized & park-facing,2018-11-06,2018-11-06T02:37:57Z
3,-38.121844,144.332476,80 Calvert Street,Hamlyn Heights,3215,NaN,3,2,1,house,"600,000",129640018,Cherished Family Home In Leafy Hamlyn Heights,2018-11-06,2018-11-06T02:00:19Z
4,-38.129071,144.323640,295 Church Street,Herne Hill,3218,bellarine_peninsula,3,1,2,house,"520,000",129459426,Potential Plus,2018-11-06,2018-11-06T02:00:17Z


In [ ]:
housing["region"].value_counts()

In [ ]:
housing["propertyType"].value_counts()

In [ ]:
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
housing['price']


In [ ]:
housing['price'] = housing['price'].astype(float)
housing.info()